In [ ]:
!pip install torch torchvision torchaudio
!pip install easyocr

In [ ]:
!pip uninstall opencv-python-headless==4.5.5.62 
!pip install opencv-python-headless==4.1.2.30

In [ ]:
import easyocr
#download the model
reader = easyocr.Reader(['tr','en'], gpu = False)

In [ ]:
# show an image
import PIL
import cv2
from PIL import ImageDraw
image_path='page3.jpg'
im = PIL.Image.open(image_path)
#im = PIL.Image.open("france.jfif")
width,height=im.size

print(width,height)

im

In [ ]:
# Doing OCR. Get bounding boxes.
bounds = reader.readtext(image_path)
bounds

In [ ]:
# Draw bounding boxes
def draw_boxes(image, bounds, color='red', width=2):
    draw = ImageDraw.Draw(image)
    for bound in bounds:
        p0, p1, p2, p3 = bound[0]
        draw.line([*p0, *p1, *p2, *p3, *p0], fill=color, width=width)
    return image

draw_boxes(im, bounds)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
print(img.shape)
points=[]
points_2=[]
points_3 = []
for bound in bounds:
    p0, p1, p2, p3 = bound[0]
    midX=int(p0[0])
    midY=int((p0[1]+p1[1]+p2[1]+p3[1])/4)
    arr=(midX,midY)
    points.append(arr)
    x1=midX/width
    y1=midY/height
    arr2=(x1,y1)
    #points_2.append(arr2)
    points_2.append((p0[0]/width,p0[0]/width))
    points_2.append((p3[0]/width,p3[0]/width))
    points_3.append((p0[1]/height,p0[1]/height))
    points_3.append((p3[1]/height,p3[1]/height))
    cv2.circle(img,(midX,midY), 10, (0,0,255), -1)

cv2_imshow(img)       

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
X=np.array(points_2)#orta noktaları numpy array'e çevirdim.

In [ ]:
#x ler için 
dbscan = DBSCAN(eps = 0.02, min_samples = 4)
model = dbscan.fit(X)
labels=model.labels_
plt.gca().invert_yaxis()
plt.scatter(X[:,0], np.zeros(len(X[:,0])),c=labels,s=80);

X_df = pd.DataFrame(X)
db_cluster = pd.DataFrame(labels)  

df = pd.concat([X_df, db_cluster], axis=1)
df.columns = ['Feature 1', 'Feature 2', 'db_cluster']
df.head()

view_cluster = df['db_cluster'].value_counts().T
view_cluster = pd.DataFrame(data=view_cluster)
view_cluster = view_cluster.reset_index()
view_cluster.columns = ['db_cluster', 'count']
view_cluster.sort_values(by='db_cluster', ascending=False)


#elde ettiğimiz clusterların centerları yazılacak.
#dikey çizgi çizdirilecek.

In [ ]:
#Centerları bul dbscan X
cluster_count=len(db_cluster.value_counts())
centers=[]
for i in range(0,cluster_count-1):
  points_of_cluster= X[labels==i,:]
  print(len(points_of_cluster))
  centroid_of_cluster = np.mean(points_of_cluster, axis=0)
  centers.append(centroid_of_cluster)
  
arr_center=np.array(centers)

plt.vlines(arr_center,0,1,color='blue')
plt.grid()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
Y=np.array(points_3)#orta noktaları numpy array'e çevirdim.

In [ ]:
#y ler için 
dbscan = DBSCAN(eps = 0.02, min_samples = 4)
model = dbscan.fit(Y)
labels_y= model.labels_
plt.gca().invert_yaxis()
plt.scatter(Y[:,0], np.zeros(len(Y[:,0])),c=labels_y,s=80);

Y_df = pd.DataFrame(Y)
db_cluster_Y = pd.DataFrame(labels_y)  

df = pd.concat([Y_df, db_cluster_Y], axis=1)
df.columns = ['Feature 1', 'Feature 2', 'db_cluster_Y']
df.head()

view_cluster = df['db_cluster_Y'].value_counts().T
view_cluster = pd.DataFrame(data=view_cluster)
view_cluster = view_cluster.reset_index()
view_cluster.columns = ['db_cluster_Y', 'count']
view_cluster.sort_values(by='db_cluster_Y', ascending=False)


In [ ]:
#Centerları bul dbscan y
cluster_count=len(db_cluster_Y.value_counts())
centers=[]
for i in range(0,cluster_count-1):
  points_of_cluster= Y[labels==i,:]
  print(len(points_of_cluster))
  centroid_of_cluster = np.mean(points_of_cluster, axis=0)
  centers.append(centroid_of_cluster)

arr_center_y=np.array(centers)
print(arr_center_y)
plt.hlines(arr_center_y,0,1,color='red')
plt.grid()
plt.show()

In [ ]:
#Aynı resim üzerinde çizimi dbscan
#Aynı resim üzerinde çizimi
plt.vlines(arr_center,0,1,color='blue')
plt.hlines(arr_center_y,0,1,color='red')
plt.grid()
plt.show()

In [ ]:
arr_lines_x=(arr_center*width)
arr_lines_y=(arr_center_y*height)
data_int_x = arr_lines_x.astype(int)
data_int_y = arr_lines_y.astype(int)

plt.vlines(data_int_x,2000,1,color='blue')
plt.hlines(data_int_y,1500,1,color='red')

plt.imshow(img)